Grupo 03

1.   Amanda Audi
2.   Carina Dourado
3.   Ingridy Silva
4.   Larissa Sayuri Santos
5.   Meyrele Torres
6.   Victor Farias



Esse código faz os seguintes passos:

10. Modelo

#Bibliotecas 

In [ ]:
import pandas as pd
import numpy as np
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt

# Leitura de Dados

In [ ]:
# Fazendo a conexao com o google drive (aula Mineracao)
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Lendo os dados
path = "XXX"
arq = "dados_processados.csv"

dados = pd.read_csv(path + arq)
#dados = pd.read_csv(arq) # pra quem não é a larissa

In [ ]:
len(dados.index)

1064

# **Criando** datasets para modelo

1. variáveis explicativas e 
2. variáveis respostas


In [ ]:
# Transformando variável tempo_tec em float
dados['tempo_tec'] = dados['tempo_tec'].str.replace (' days', '')
dados['tempo_tec'] = dados['tempo_tec'].astype(float)

In [ ]:
#dados.columns
dados = dados.dropna() # tive que tirar os NaN porque tava dando erro no modelo
len(dados.index)

1050

In [ ]:
# Esse eh o código do Edu da aula 01 (prefeitos)
var_resposta = dados[['demissao']]

var_explicativas = dados[['time', 'tecnico', # quali nominais
                          'colocacao',
                          # result. jogo - quant discreta
                          'gols', 'total_gols', 'gols_outro_time', 'dif_gols',
                          # status jogo - binárias
                          'derrota', 'empate', 'vitoria', 
                          # histórico acumulado no ano - quant discreta
                          'vit_acum_tot', 'emp_acum_tot', 'der_acum_tot', 
                          # desempenho a curto prazo - quant discreta
                          'der_acum_ult5', 'emp_acum_ult5', 'vit_acum_ult5', 
                          # outras características do técnico 
                          'Interino', 'tempo_tec']]

# Transformar variáveis categóricas em numéricas



In [ ]:
var_explicativas = pd.get_dummies(var_explicativas)
var_explicativas

,colocacao,gols,total_gols,gols_outro_time,dif_gols,derrota,empate,vitoria,vit_acum_tot,emp_acum_tot,der_acum_tot,der_acum_ult5,emp_acum_ult5,vit_acum_ult5,Interino,tempo_tec,time_Atlético-MG,time_Botafogo,time_Corinthians,time_Cruzeiro,time_Flamengo,time_Grêmio,time_Internacional,time_Palmeiras,time_Sport Recife,time_São Paulo,tecnico_Abel Braga,tecnico_Abel Ferreira,tecnico_Adilson Batista,tecnico_Alberto Valentim,tecnico_Andrey Lopes,tecnico_André Jardine,tecnico_Bruno Lazaroni,tecnico_Claudinei Oliveira,tecnico_Coelho,tecnico_Cuca,tecnico_Daniel Paulista,tecnico_Diego Aguirre,tecnico_Domènec Torrent,tecnico_Dorival Júnior,tecnico_Eduardo Baptista,tecnico_Eduardo Barroca,tecnico_Eduardo Coudet,tecnico_Emiliano Ramón Díaz,tecnico_Fernando Diniz,tecnico_Flávio Tenius,tecnico_Fábio Carille,tecnico_Jair Ventura,tecnico_Jorge Jesus,tecnico_Jorge Sampaoli,tecnico_Levir Culpi,tecnico_Luiz Felipe Scolari,tecnico_Lúcio Flávio,tecnico_Mano Menezes,tecnico_Marcelo Salles,tecnico_Marcos Paquetá,tecnico_Maurício Barbieri,tecnico_Milton Mendes,tecnico_Nelsinho Baptista,tecnico_Odair Hellmann,tecnico_Osmar Loss,tecnico_Paulo Autuori,tecnico_Renato Gaúcho,tecnico_Ricardo Colbachini,tecnico_Rodrigo Santana,tecnico_Roger Machado,tecnico_Rogério Ceni,tecnico_Thiago Larghi,tecnico_Tiago Nunes,tecnico_Vanderlei Luxemburgo,tecnico_Vizolli,tecnico_Vágner Mancini,tecnico_Wesley Carvalho,tecnico_Zé Ricardo
0,13.0,1.0,3.0,2.0,-1.0,1,0,0,0,0,1,1,0,0,0,65.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,13.0,2.0,3.0,1.0,1.0,0,0,1,1,0,1,1,0,1,0,72.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,11.0,1.0,1.0,0.0,1.0,0,0,1,2,0,1,1,0,2,0,79.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,4.0,2.0,4.0,2.0,0.0,0,1,0,2,1,1,1,1,2,0,86.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,6.0,2.0,3.0,1.0,1.0,0,0,1,3,1,1,1,1,3,0,93.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1058,3.0,1.0,3.0,2.0,-1.0,1,0,0,16,10,7,3,2,0,0,493.0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1059,4.0,1.0,2.0,1.0,0.0,0,1,0,16,11,7,2,3,0,1,18.0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1060,4.0,1.0,2.0,1.0,0.0,0,1,0,16,12,7,2,3,0,1,9.0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1061,4.0,2.0,3.0,1.0,1.0,0,0,1,17,12,7,1,3,1,1,13.0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


# Criar o modelo a partir da nossa base de treinamento


In [ ]:
from sklearn.linear_model import LogisticRegression

logisticRegr = LogisticRegression(max_iter = 2000) # aumentei o número de iterações pq tava dando erro

model = logisticRegr.fit(var_explicativas, var_resposta.values.ravel())

In [ ]:
model

LogisticRegression(max_iter=2000)

# Previsoes na base de TREINO

In [ ]:
predictions = logisticRegr.predict(var_explicativas)
predictions_prob = logisticRegr.predict_proba(var_explicativas)


predictions = pd.DataFrame(predictions, columns = ['predito'])
predictions_prob = pd.DataFrame(predictions_prob, columns=["Prob_0", "Prob_1"])
predictions_prob['demissao'] = dados['demissao']
predictions_prob

,Prob_0,Prob_1,demissao
0,0.924952,0.075048,0.0
1,0.975723,0.024277,0.0
2,0.983039,0.016961,0.0
3,0.993790,0.006210,0.0
4,0.994305,0.005695,0.0
...,...,...,...
1045,0.859186,0.140814,0.0
1046,0.857512,0.142488,0.0
1047,0.860781,0.139219,0.0
1048,0.911812,0.088188,0.0


In [ ]:
# Atualizando dados com predição do modelo
dados['Prob_1'] = predictions_prob['Prob_1']
dados_sort = dados.sort_values(by = 'Prob_1', ascending = False)
dados_sort.head(15)

,index,ano,data,rodada,time,tecnico,colocacao,gols,total_gols,gols_outro_time,dif_gols,derrota,empate,vitoria,der_acum_ult5,emp_acum_ult5,vit_acum_ult5,vit_acum_tot,emp_acum_tot,der_acum_tot,id_tecnico,id_tecnico_lagged,demissao,Comeco,Interino,tempo_tec,Prob_1
566,6615,2019,2019-11-27,35,Flamengo,Jorge Jesus,1.0,4.0,5.0,1.0,3.0,0,0,1,0,1,4,26,6,3,5.0,5.0,0.0,2019-06-13,0,167.0,0.425033
774,6796,2020,2020-10-11,15,Botafogo,Bruno Lazaroni,17.0,2.0,3.0,1.0,1.0,0,0,1,0,3,2,3,9,3,6.0,6.0,0.0,2020-10-01,0,10.0,0.350416
303,6268,2018,2018-12-02,38,Palmeiras,Luiz Felipe Scolari,1.0,3.0,5.0,2.0,1.0,0,0,1,0,1,4,23,11,4,3.0,3.0,0.0,2018-08-03,0,121.0,0.314128
631,6501,2019,2019-10-10,24,Internacional,Odair Hellmann,6.0,0.0,1.0,1.0,-1.0,1,0,0,2,2,1,11,5,8,1.0,2.0,1.0,2017-11-14,0,695.0,0.292779
973,6882,2020,2020-12-05,24,Palmeiras,Abel Ferreira,4.0,2.0,4.0,2.0,0.0,0,1,0,1,1,3,10,9,5,7.0,7.0,0.0,2020-11-03,0,32.0,0.261068
629,6482,2019,2019-09-29,22,Internacional,Odair Hellmann,5.0,1.0,2.0,1.0,0.0,0,1,0,1,1,3,11,4,7,1.0,1.0,0.0,2017-11-14,0,684.0,0.253471
441,6500,2019,2019-10-10,24,Botafogo,Bruno Lazaroni,12.0,3.0,4.0,1.0,2.0,0,0,1,4,0,1,9,3,12,5.0,5.0,0.0,2019-10-07,1,3.0,0.248759
488,6596,2019,2019-11-17,33,Corinthians,Coelho,8.0,0.0,0.0,0.0,0.0,0,1,0,2,2,1,12,14,7,4.0,4.0,0.0,2019-11-04,1,13.0,0.241207
678,6594,2019,2019-11-17,33,Palmeiras,Mano Menezes,2.0,1.0,2.0,1.0,0.0,0,1,0,0,2,3,19,11,3,4.0,4.0,0.0,2019-09-03,0,75.0,0.240949
789,6943,2020,2021-01-17,30,Botafogo,Eduardo Barroca,19.0,1.0,3.0,2.0,-1.0,1,0,0,4,0,1,4,11,15,9.0,9.0,0.0,2020-11-27,0,51.0,0.233021


In [ ]:
predictions.value_counts()

predito
0.0        1050
dtype: int64

# Qualidade do modelo

In [ ]:
# Métricas 
metrics_list = ['min', 'mean', 'median', 'max', 'std']
predictions_prob.agg({'Prob_1': metrics_list})

,Prob_1
min,0.000642
mean,0.041929
median,0.024259
max,0.425033
std,0.049518


In [ ]:
# Quantis
predictions_prob["Prob_1"].quantile([.01, .05, .1, .25, .5, .75, .9, .95, .99])

0.01    0.001486
0.05    0.003213
0.10    0.004516
0.25    0.009793
0.50    0.024259
0.75    0.054438
0.90    0.106234
0.95    0.148981
0.99    0.222750
Name: Prob_1, dtype: float64

In [ ]:
import altair as alt              # histograma e scatterplot

alt.Chart(predictions_prob).mark_point().encode(
        alt.X('Prob_1',
          title = 'Probabilidade de demissão'),
        alt.Y('demissao',
          title = 'Demissão')).properties(height = 320, width = 640).interactive()

alt.Chart(...)

In [ ]:
# A partir daqui não sei muito bem o que é pra fazer...
ponto_corte = 0.9

predictions_prob[['pred_eleger']] = np.where(analise[['Prob_1']] >= ponto_corte, 1, 0)

predictions_prob[['pred_eleger']].sum()

Prob_1    24.0
dtype: float64

In [ ]:
y_true = pd.Series(list(map(int, analise['eleito'])), name = 'Actual')
y_pred = pd.Series(list(map(int, analise['pred_eleger'])), name = 'Predicted')


df_confusion = pd.crosstab(y_true, y_pred, rownames=['Actual'], colnames=['Predicted'], margins=True)
df_confusion